# Module 1: Kubernetes Storage Overview

## Exercise 1: Working with Kubernetes Storage Volumes

In this exercise, you explore native Kubernetes storage objects, including emptyDir, hostPath, and NFS
volume types. 

You also create a manual persistent volume (PV) and persistent volume claim (PVC).

**Objectives**

This exercise focuses on enabling you to do the following:
  - Set up an emptyDir volume
  - Configure a hostPath volume
  - Deploy a pod with two containers that share storage
  - Configure an NFS server by using NetApp ONTAP software
  - Set up an NFS volume
  - Configure a PV and a PVC


**Exercise Equipment**

In this exercise, you use the following systems.

| System                  | Host Name   | IP Addresses   | User Name (case sensitive) | Password  |
|-------------------------|-------------|----------------|----------------------------|-----------|
| Linux Mint 20           | jumphost    | 192.168.0.5    | user                       | Netapp1!  |
| Kubernetes Worker 1     | kubwor1-1   | 192.168.0.62   | root                       | Netapp1!  |
| Kubernetes Worker 2     | kubwor1-2   | 192.168.0.63   | root                       | Netapp1!  |
| Kubernetes Worker 3     | kubwor1-3   | 192.168.0.64   | root                       | Netapp1!  |

**Prerequisites**

Before starting this exercise, you should take the following actions:
  - Set up your Integrated Development Environment (IDE)
  - Download the courseware GIT repository
  - Configure your IDE to have access to your Kubernetes clusters


#### Task 1: Set up an emptyDir volume

In this task, you create the simplest volume option available in Kubernetes: **emptyDir**.


---

All files in this exercise have the relative path of Exercise 1. 

If you use kubectl command via a terminal window, change directory to the Exercise 1 directory

---

Review and update the [exercise1Task1.yaml](./exercise1Task1.yaml) file with the following information,

and then save the file as **exercise1Task1mod.yaml**.

```yaml
volumeMounts:
- mountPath: /opt/this
  name: myvol
volumes:
- name: myvol
  emptyDir: {}


---

Create an instance of the exercise1Task1mod.yaml file:


In [ ]:
kubectl create -f exercise1Task1mod.yaml


---

Describe the pod again and review the events:


In [ ]:
kubectl describe pod emptydir-pod


---

Answer the following question:

Is the pod running?


---

Connect to the pod:


In [ ]:
kubectl exec  emptydir-pod -- ls -la /opt/this

---

From the container’s prompt, list the directory of the /opt/this directory.


---

From the container’s prompt, create a file with the touch command in the /opt/this
directory.


In [ ]:
kubectl exec emptydir-pod -- touch /opt/this/file1

---

From the container’s prompt, list the directory of the /opt/this directory.


In [ ]:
kubectl exec  emptydir-pod -- ls -la /opt/this

---

Exit the container’s prompt.


---

Delete the emptyDir pod:


In [ ]:
kubectl delete pod emptydir-pod


---

This course includes CHALLENGE STEPS to help you to explore deeper sections of
the Kubernetes ecosystem. 

NOTE: These CHALLENGE STEPS are optional, but you cannot skip steps if you want to complete the entire challenge.


---

CHALLENGE STEP: Re-create the pod.

Does the file that you created in still exist?


In [ ]:
kubectl create -f exercise1Task1mod.yaml


In [ ]:
kubectl get pod emptydir-pod -o wide


In [ ]:
kubectl exec  emptydir-pod -- ls -la /opt/this

---

CHALLENGE STEP: Exit the container’s prompt and delete the second instance of the
emptyDir pod.


In [ ]:
kubectl delete pod emptydir-pod


---
---

#### Task 2: Configure a hostPath volume

In this task, you create a Debian container and provide access to a local mount point on the hosting worker node.


---

Review and update the [exercise1Task2-1.yaml](./exercise1Task2-1.yaml) file with the following definitions, 

and then save the file as **exercise1Task2-1mod.yaml**.


```yaml
spec: 
  volumes:
  - name: local-vol
    hostPath:
      path: /hostVol
      type: Directory
  Containers: 
    volumeMounts:
    - mountPath: /root/
      name: local-vol


---


You can find the solution in the subfolder.


---

Create an instance of the exercise1Task2-1mod.yaml file:


In [ ]:
kubectl create -f exercise1Task2-1mod.yaml


---

Identify which node the pod is running on by using the 

`kubectl get pods -o wide` 

command.


---

In [ ]:
kubectl get pods -o wide -w

Describe the pod, review the events, and answer the following questions:

Is the pod running? Why or why not?


In [ ]:
kubectl describe pod hostpath-pod

---

Using a new terminal window, open a Secure Shell (SSH) session to the worker node that is assigned to the pod:

ssh root@192.168.0.6x

Create the required directory on the worker:

`mkdir /hostVol`

In [ ]:
pod=$(kubectl get pods -o wide | awk 'NR==2{print $7}');ssh root@$pod "mkdir -p /hostVol; ls -d /hos*"


---

From the worker node’s prompt, create a local file in the directory:


In [ ]:

ssh root@$pod "touch /hostVol/exercise1Task2; ls /hostVol/"


---

Wait a few minutes for the pod to identify the new directory.


---

Describe the pod again, and review the events:


In [ ]:

kubectl describe pod hostpath-pod

---

Answer the following question:

Is the pod running?


---

The pod should be running. 

If the pod is not running, delete the pod and re-create it.


---

Connect to the pod and list the contents of the /root/ directory:


In [ ]:
kubectl exec hostpath-pod -- ls -l /root/

Answer the following question:

Do you see the file that you created from the local worker node?


---

From the container’s prompt, create a file in the /root directory.


In [ ]:
kubectl exec hostpath-pod -- /bin/sh -c "touch /root/file2; ls -l /root"

---

Answer the following question:

Do you see the file that you created in the `hostpath-pod` on the local worker node /hostVol dir?


In [ ]:
ssh root@$pod ls -l /hostVol

---

Answer the following question:

What happens to file access if the pod is destroyed and re-created on the other worker node?


---

Delete the pod:



In [ ]:
kubectl delete pod hostpath-pod


---

Copy the completed YAML file and rename the copy as `exercise1Task2-2.yaml`


In [ ]:
cp ./exercise1Task2-1mod.yaml ./exercise1Task2-2.yaml

---

Change the following in the [exercise1Task2-2.yaml](./exercise1Task2-2.yaml) file and then save the file.

```yaml
Pod name: hostpath2-pod
hostPath:
  path: /hostVol2
  type: DirectoryOrCreate


---

Create an instance of the ``exercise1Task2-2.yaml` file:




In [ ]:
kubectl create -f exercise1Task2-2.yaml


---

Describe the pod again, and review the events:



In [ ]:
kubectl describe pod hostpath2-pod


---

Answer the following question:

Is the pod running?


In [ ]:
kubectl get pod hostpath2-pod -o wide

---

Notice that the /hostVol2 was created on the appropriate worker node.


In [ ]:
pod2=$(kubectl get pod hostpath2-pod -o wide|awk 'NR==2{print $7}'); ssh root@$pod2 "ls -ld /host*"

---

Connect to the pod and 

From the container’s prompt, list the directory of the /root directory.

In [ ]:
kubectl exec -it hostpath2-pod -- /bin/sh -c 'ls -l /root'


---

From the container’s prompt, create a file in the mounted path.


In [ ]:
kubectl exec -it hostpath2-pod -- /bin/sh -c 'touch /root/filehostpath2; ls -l /root'


---

Delete the hostpath2-pod pod.


In [ ]:
kubectl delete pod hostpath2-pod

---

CHALLENGE STEP: 

Delete and re-create the pod, 

see which node is selected to host the second instance of the pod, 

and answer the following question:

What happens to the hostPath directory on the local worker node?


---
---

#### Task 3: Deploy a pod with two containers that share storage

In this task, you define a volume named shared-data. 

The volume’s type is emptyDir. 

The first container runs an NGINX server and mounts the shared volume to the directory `/usr/share/nginx/html`. 

The second container is based on the Debian image. 

The second container mounts the shared volume to the directory /pod-data, which runs a loop that writes the
current date and time to the `index.html` file in the shared volume. 

The container waits 10 seconds before repeating the loop. 

This task uses nodeport service. 

For more information regarding this service type, please see Kubernetes Administration.


---

Review and update the [exercise1Task3.yaml](./exercise1Task3.yaml) file with the following definitions, 

and then save the file as **exercise1Task3mod.yaml**.

```yaml

spec.volumes
  - name: shared-vol
  
Add an emptyDir volume definition.

In both containers’ volumeMounts definition, use the name shared-vol.


---

You can find the solution in the subfolder.

<details> <summary> Solution  </summary> 

[exercise1Task3.yaml](./Solutions/exercise1Task3.yaml) 

```yaml

---

Create an instance of the exercise1Task3mod.yaml file:


In [ ]:
kubectl create -f exercise1Task3mod.yaml


---

Connect to the first container of the pod

From the container’s prompt, 

verify that the index.html page is being updated every second:

```bash
# tail /usr/share/nginx/html/index.html

```



In [ ]:
kubectl exec -it two-pod -c first -- /bin/sh -c "tail /usr/share/nginx/html/index.html "


---

Create a service that enables a worker node to access the two-pod:


In [ ]:
kubectl expose pod two-pod --type=NodePort --port=80


---

Identify the service node port:


In [ ]:
kubectl describe service two-pod


---

Open a web browser to the following URL: 

http://[a worker node’s IP address]:3xxxx [node port from previous step].


Sample output:
```
Fri Feb 25 17:05:43 UTC 2024 Hello from the second container
Fri Feb 25 17:05:44 UTC 2024 Hello from the second container
Fri Feb 25 17:05:45 UTC 2024 Hello from the second container
Fri Feb 25 17:05:46 UTC 2024 Hello from the second container
```


In [ ]:
nodeport=$(kubectl describe service two-pod|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

---


Delete the pod named two-pod and remove the service. 

HINT: To remove a service, use

`kubectl delete service [name of service]`.


In [ ]:
kubectl delete service two-pod

In [ ]:
kubectl delete pod two-pod

#### Task 4: Configure an NFS server by using NetApp ONTAP software

In this task, you configure a storage VM (storage virtual machine, also known as SVM) for NFS protocols. 

[Method 1](#method1) 

You use an open-source tool called `gateway`  to create this SVM. 

**OR**

[Method 2](#method2)

[Method 2](#section2)



---

#<a id="section1"></a>

<a id="method1"></a>

##### Method 1

Review and execute the [exercise1Task4-1.yaml](./exercise1Task4-1.yaml) file to create the gateway operator. 

For more information, on the non-NetApp supported community project, see this:

https://github.com/NetApp-Learning-Services/gateway.


In [ ]:
kubectl create -f exercise1Task4-1.yaml



- Verify that you created the gateway-system namespace and that the operator pod is running in that namespace.



- Review and execute the [exercise1Task4-2.yaml](./exercise1Task4-2.yaml) file to create an SVM called svm0 with the defined protocol in ONTAP Cluster 1.



In [ ]:
kubectl create -f exercise1Task4-2.yaml


CHALLENGE STEP: Review the logs of the manager container for the gateway-manager deployment’s pod to see the gateway operator in action:

`kubectl -n gateway-system logs gateway-operator-[unique id] -c manager`

In [ ]:
go=$(kubectl -n gateway-system get pods| grep gateway-operator | awk '{print $1}')
kubectl -n gateway-system logs $go -c manager


---

Open a browser and go to https://192.168.0.101/

(which is your Cluster1 management LIF’s address).

NOTE: You might need to approve the security warning.


---

For now, use the standard System Manager. Click the link: Not now. 

Sign in to System Manager. 

Skip this step if you already have selected System Manager instead of using NetApp BlueXP.


---

Authenticate with your ONTAP cluster by providing the following credentials:
  - Login as: admin
  - Password: Netapp1!


---

Click Sign In.

---

From the left pane, navigate to Storage > Storage VMs.


---

Review the settings of svm0 and verify that NFS is configured. 

Also verify that the Cluster1_01_FC_1 is an available local tier for this SVM by clicking Edit on the SVM.


---
---

Remember: Skip the following steps if you already executed Method 1.

[Jump Ahead](#svmcreated) to `SVM Created`

##<a id="method2"></a>

##### Method 2

Manually create the SVM svm0 using System Manager

---

Authenticate with your ONTAP cluster (https://192.168.0.101) by providing the following credentials:
  - Login as: admin
  - Password: Netapp1!

---

Click Sign In.

---

From the left pane, navigate to Storage > Storage VMs.

---

Click Add to start the wizard to create an SVM.

---

Enter the following information:
  - Storage VM name: svm0
  - Select the tab SMB/CIFS, NFS, S3.
  - Access protocol: Select Enable NFS.
  - Enable NFS client access: Selected
  
  - Rule: Add a rule:
    - Client specification: 0.0.0.0/0
    - Protocols: NFS (both v3 and v4)
    - Read-only: All selected
    - Read/write: All selected
    - Superuser: All selected
    - Click Save.
  
  - Under Network Interface:
    - IP address: 192.168.0.31
    - Subnet mask: 255.255.255.0
    - Broadcast domain: Default

---

Continue by entering the following information:

Manage administrator account: Selected
  - User name: vsadmin
  - Password: Netapp1!
  - Verify password: Netapp1!
  - Select Add a network interface for storage VM management
    - IP address: 192.168.0.30
    - Subnet mask: 24
    - Broadcast domain: Default

---

At the bottom of the dialog box, click Save.
You should see svm0 in the list of SVMs.

---

Click the newly created svm0 link.

You should see the Overview page of svm0.

---

Click the Edit button in the upper-right corner of the Overview page.

---

Select the “Resource Allocation” checkbox to limit volume creation to preferred local tiers, and make sure that Cluster1_01_FC_1 is selected in the list of local tiers.

---

Click Save.

---


<a id="svmcreated"></a>



##### SVM Created
This is the end of the SVM creation steps. 

You should now have a svm0 in cluster1. 

You will now continue along with the setting up the NFS share.


---
---




In ONTAP System Manager, navigate to Storage > Volumes.


---

Click Add to create a volume.


---

Add the following details:
 - Name: nfs
 - Size: 1GiB
 
Click the More button.
  
  Ensure that the “Export via NFS” checkbox is selected.
  
  Ensure that the default rule has all NFS protocols selected with   the clients of 0.0.0.0/0.


---

Click Save to create the volume.


---

From the list of volumes, identify and select the link for the new nfs volume.


---

On the nfs volume's Overview page, click the Edit button in the upper-right corner.


---

In the details of the volume, perform the following:
 - Ensure that the security type is UNIX.
 - Under UNIX Permissions, select Read, Write, and Execute so that all checkboxes are
selected.

---

Click Save to complete the volume’s edit.


---
---

#### Task 5: Set up an NFS volume

In this task, you create a pod that directly connects to the NFS export that you created in [Task 4](#task-4-configure-an-nfs-server-by-using-netapp-ontap-software) of this exercise. 

---
---

NOTE: Every node in the Kubernetes cluster needs to have the `nfs-common` package
installed to appropriately mount an NFS export for a container.

---

Review and update the [exercise1Task5.yaml](./exercise1Task5.yaml) file with the following definitions, and then
save the file as `exercise1Task5mod.yaml`.

```
spec.volumes[0].name: nfsvol
spec.volumes[0].nfs.server: 192.168.0.31
spec.volumes[0].nfs.path: /nfs
spec.volumes[0].nfs.readOnly: false
```

---
You can find the solution in the subfolder.

<details> <summary> Solution  </summary> 

[exercise1Task5.yaml](./Solutions/exercise1Task5.yaml)
  ```yaml

---

Create an instance of the `exercise1Task5mod.yaml` file:


In [ ]:
kubectl create -f exercise1Task5mod.yaml


---

Connect to the alpine pod that you created:

`kubectl exec direct-nfs-pod -it -- /bin/sh`


In [ ]:

kubectl exec direct-nfs-pod -it -- /bin/sh


In a terminal window (```CTRL+` ```), you connect to the container and execute the commands. You can copy and paste the commands below in the terminal

---

From the container’s prompt, change the directory to volume mount:
```bash
# cd /opt/this
```

---

From the container’s prompt, create a file and perform other operations at this location to
demonstrate that you have read/write permission:

```bash
# echo "<html><body>hello</body></html>" > index.html
# ls
# cat index.html
```

---

Use Ctrl-D to exit the container’s prompt.


---

In [ ]:
kubectl exec  direct-nfs-pod -- /bin/sh -c "echo '<html><body>hello direct-nfs-pod </body></html>' > /opt/this/index.html; ls /opt/this/; cat /opt/this/index.html"

Delete the pod:


In [ ]:
kubectl delete pod direct-nfs-pod


---

In the ONTAP System Manager, under the **nfs volume** page, you can 

select the **Explorer** tab under the **File System** tab 

to view the folders and files that you created in the pod. 

The data is persisted in the ONTAP cluster.


---
---

#### Task 6: Configure a PV and a PVC

In this task, you create a PV that is manually attached to the NFS export that you created in Task 4.

Then, you bind that PV to a PVC. 

Finally, you use the PVC in a pod so that the pod’s container can access the NFS export. 

Later, you use NetApp Astra Trident to automate this process of creating PVs.


---

Review and update the [exercise1Task6-1.yaml](./exercise1Task6-1.yaml) file with the following PV specification
definitions, and then save the file as `exercise1Task6-1mod.yaml`.

Add an nfs definition with the following values:

```
nfs.server: 192.168.0.31
nfs.path: /nfs
nfs.readOnly: false
```

You can find the solution in the subfolder.

<details> <summary> Solution</summary>

[exercise1Task6-1.yaml](./Solution/exercise1Task6-1.yaml)
  ```yaml

apiVersion: v1
kind: PersistentVolume
metadata:
  name: manual-nfs-pv
  labels:
    storage.k8s.io/name: nfs
spec:
  accessModes:
    - ReadWriteOnce
    - ReadOnlyMany
    - ReadWriteMany
  capacity:
    storage: 1Gi
  storageClassName: ""
  persistentVolumeReclaimPolicy: Recycle
  volumeMode: Filesystem
  nfs:
    server: 192.168.0.31
    path: /nfs
    readOnly: false
    

---

Create an instance of the PV in the `exercise1Task6-1mod.yaml` file:


In [ ]:
kubectl create -f exercise1Task6-1mod.yaml


---

Verify that you created the manual-nfs-pv PV:


In [ ]:
kubectl get pv


---

Describe the manual-nfs-pv PV:


In [ ]:
kubectl describe pv manual-nfs-pv


---

Review and update the [exercise1Task6-2.yaml](./exercise1Task6-2.yaml) file with the following PVC spec
definitions, 

and then save the file as `exercise1Task6-2mod.yaml`.

```
spec.accessModes[0]: ReadWriteMany
spec.storageClassName: '' # empty string
spec.volumeName: [name of the PV from `exercise1Task6-2mod.yaml`]
spec.resources.requests.storage: 1Gi
```

You can find the solution in the subfolder.

<details> <summary> Solution </summary>

[exercise1Task6-2.yaml](./Solutions/exercise1Task6-2.yaml)

  ```yaml
  apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: manual-nfs-pvc
  labels:
    storage.k8s.io/name: nfs
spec:
  accessModes:
    - ReadWriteMany
  storageClassName: ''  # empty string otherwise default storage class is used
  volumeName: manual-nfs-pv
  resources: 
    requests:
      storage: 1Gi

Create an instance of the PVC in the `exercise1Task6-2mod.yaml` file:


In [ ]:
kubectl create -f exercise1Task6-2mod.yaml


---

Verify that you created the manual-nfs-pvc PVC:


In [ ]:
kubectl get pvc


---

Describe the manual-nfs-pvc PVC:


In [ ]:
kubectl describe pvc manual-nfs-pvc


---

Ensure that the PVC’s status is **bound**, which means that the PVC was mapped to the PV.


---

Review and update the [exercise1Task6-3.yaml](./exercise1Task6-3.yaml) file with the following Pod volumes
definitions, 

and then save the file `exercise1Task6-3mod.yaml`.

```
  - name: nfs-storage
  - persistentVolumeClaim:claimName: [name of the PVC you created in exercise1Task6-2.yaml]
```

<details> <summary>Solution  </summary>

[exercise1Task6-3.yaml](./Solutions/exercise1Task6-3.yaml)

```yaml
kind: Pod
apiVersion: v1
metadata:
  name: manual-nfs-pod
  labels:
    app: nfs-web
spec:
  containers:
    - name: nfs-container
      image: nginx:1.25-alpine-slim
      imagePullPolicy: IfNotPresent
      resources:
        requests:
          memory: "64Mi"
          cpu: "250m"
        limits:
          memory: "128Mi"
          cpu: "500m"
      ports:
        - containerPort: 80
          name: "http-server"
      volumeMounts:
        - mountPath: "/usr/share/nginx/html"
          name: nfs-storage
  volumes:
  - name: nfs-storage
    persistentVolumeClaim:
      claimName: manual-nfs-pvc
      

---

Create an instance of the PVC in the `exercise1Task6-3mod.yaml` file:


In [ ]:
kubectl create -f exercise1Task6-3mod.yaml


---

Verify that you created the manual-nfs-pod pod:


In [ ]:
kubectl get pod


---

Describe the manual-nfs-pod pod:


In [ ]:
kubectl describe pod manual-nfs-pod


---

Ensure that the pod is using the PV that is backed by the NFS export in svm0.


In [ ]:
kubectl describe pod manual-nfs-pod | grep -A5 -i volume


---

CHALLENGE STEP: 

Open an shell to the manual-nfs-pod, 

and navigate to the mount path of the PV. 

Verify that the index.html file that you created in the previous 
task is available to the NGINX server.



In [ ]:
kubectl exec  manual-nfs-pod -- /bin/sh -c "ls /usr/share/nginx/html/; cat /usr/share/nginx/html/index.html"

---

CHALLENGE STEP: 

Create a NodePort service for the manual-nfs-pod 

and, in a browser, open the index.html file that you created.


In [ ]:
kubectl expose pod manual-nfs-pod --type=NodePort --name=manual-nfs-service


In [ ]:
kubectl describe service manual-nfs-service

In [ ]:
nodeport=$(kubectl describe service manual-nfs-service|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

---

Delete the manual-nfs-pod pod:


In [ ]:
kubectl delete pod manual-nfs-pod


---
---

End of exercise
